## Objective

download all genomes in NCBI refseq or genbank at or below a specified taxon_id

## Materials, Methods, and Functions

In [19]:
import Pkg
# Pkg.instantiate()
import Mycelia
import ProgressMeter
import DataFrames
import Downloads

In [3]:
# default parameters
taxon_id = 10239
data_dir = "../../data/genomes"

"../../data/genomes"

In [25]:
# Parameters
taxon_id = 10239
data_dir = "/workspaces/Mycelia/data/genomes"

"/workspaces/Mycelia/data/genomes"

In [27]:
done_file = "$data_dir/done.txt"
if isfile(done_file)
    rm(done_file)
end

In [ ]:
# # use taxonkit to get the list of all phylogenetic classifications
# # at or below the parameterized taxon_id
# taxonkit list --ids 10239 --indent ""
# taxonkit list --ids $(taxon_id) --indent ""

In [13]:
selected_taxids = Set([parse(Int, line) for line in eachline(open(`taxonkit list --ids $(taxon_id) --indent ""`)) if !isempty(line)])

Set{Int64} with 228892 elements:
  656260
  1742422
  1427858
  2783550
  1599415
  300190
  2901373
  468417
  864692
  1149619
  2798297
  2764961
  2827923
  1431632
  1733762
  1924399
  1613832
  2558856
  2822896
  1686811
  438298
  438823
  1347503
  751008
  1205489
  ⋮ 

In [12]:
# use mycelia function to get the list of reference genomes
# todo make this a dynamic "load_ncbi_metadata" function
ncbi_metadata = Mycelia.load_refseq_metadata()

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category
,String,String,String,String,String
1,GCF_000001215.4,PRJNA164,SAMN02803731,,reference genome
2,GCF_000001405.40,PRJNA168,,,reference genome
3,GCF_000001635.27,PRJNA169,,,reference genome
4,GCF_000001735.4,PRJNA116,SAMN03081427,,reference genome
5,GCF_000001905.1,PRJNA70973,SAMN02953622,AAGU00000000.3,representative genome
6,GCF_000001985.1,PRJNA32665,SAMN02953685,ABAR00000000.1,representative genome
7,GCF_000002035.6,PRJNA13922,SAMN06930106,,reference genome
8,GCF_000002075.1,PRJNA209509,SAMN02953658,AASC00000000.3,representative genome
9,GCF_000002235.5,PRJNA13728,SAMN00829422,AAGJ00000000.6,representative genome


In [14]:
taxid_match = map(taxid -> taxid in selected_taxids, ncbi_metadata[!, "taxid"])
species_taxid_match = map(taxid -> taxid in selected_taxids, ncbi_metadata[!, "species_taxid"])
joint_match = taxid_match .| species_taxid_match
filtered_ncbi_metadata = ncbi_metadata[joint_match, :]

,# assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid
,String,String,String,String,String,Int64
1,GCF_000819615.1,PRJNA485481,,,na,2886930
2,GCF_000820355.1,PRJNA485481,,,na,191289
3,GCF_000820495.2,PRJNA485481,,,na,518987
4,GCF_000836805.1,PRJNA485481,,,reference genome,90963
5,GCF_000836825.1,PRJNA485481,,,na,1891754
6,GCF_000836845.1,PRJNA485481,,,na,196399
7,GCF_000836865.1,PRJNA485481,,,na,10570
8,GCF_000836885.1,PRJNA485481,,,na,57579
9,GCF_000836905.1,PRJNA485481,,,na,2905681


In [28]:
# download them all
# for extension in ["genomic.fna.gz", "protein.faa.gz", "genomic.gbff.gz"]
extension = "genomic.fna.gz"
# outdir = mkpath(joinpath(DIR, extension))
ProgressMeter.@showprogress for row in DataFrames.eachrow(filtered_ncbi_metadata)
    url = Mycelia.ncbi_ftp_path_to_url(row["ftp_path"], extension)
    outfile = joinpath(data_dir, basename(url))
    if !isfile(outfile)
        try
            Downloads.download(url, outfile)
        catch e
            # @show e
            showerror(stdout, e)
            # @assert extension == "protein.faa.gz"
            # here is where we should call prodigal to fill in protein annotations if we don't otherwise see them
        end
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:22:53m


## Experimental/Simulated Observations

In [4]:
mkpath("$data_dir/genomes")
touch("$data_dir/done.txt")

"/workspaces/Mycelia/data/genomes/done.txt"

## Analysis, Statistics, and Visualizations

## Summary of Results

## Conclusions and Future Directions